# Tutorial: Turbine Unit Model with IAPWS Property Package



![](expander_2.svg)

In this tutorial, we will simulate steam flow through a turbine unit model, using the IAPWS property package.
The inlet specifications are as follows:

* Flow Rate = 1 kmol/hr
* Mole fraction (H2O) = 1
* Pressure = 101325 Pa
* Temperature = 390 K 

The inlet specifications are as follows:
* Pressure Decrease = 50000 Pa
* Isentropic Efficiency = 0.9

## Setting up the problem in IDAES

In [1]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, value

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) # dynamic or ss flowsheet needs to be specified here


# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.generic_models.properties import iapws95
from idaes.generic_models.properties.helmholtz.helmholtz import (
    PhaseType
)

# Add properties parameter block to the flowsheet with specifications
m.fs.properties = iapws95.Iapws95ParameterBlock(default={"phase_presentation":PhaseType.G})

In [2]:
# Import turbine unit model from the model library
from idaes.generic_models.unit_models.pressure_changer import Turbine

# Create an instance of the turbine unit, attaching it to the flowsheet
# Specify that the property package to be used with the pump is the one we created earlier.
m.fs.turbine = Turbine(default={"property_package": m.fs.properties})

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


In [3]:
assert DOF_initial == 5

In [4]:
# Fix the stream inlet conditions
m.fs.turbine.inlet.flow_mol[0].fix(1*1000/3600) # converting to mol/s as unit basis is mol/s
m.fs.turbine.inlet.enth_mol[0].fix(iapws95.htpx(T=390, P=101325))
m.fs.turbine.inlet.pressure[0].fix(101325)
# Fix stream outlet and pump conditions
m.fs.turbine.deltaP.fix(-50000)
m.fs.turbine.efficiency_isentropic.fix(0.9)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

The final DOF is 0


In [5]:
assert DOF_final == 0

### Flowsheet Initialization

In [6]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.turbine.initialize(outlvl=idaeslog.WARNING)
# From the output it can be inferred that since there are no errors or warnings encountered during initialization, nothing is displayed  

### Obtaining Simulation Results

In [7]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [8]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert solve_status.solver.termination_condition == TerminationCondition.optimal
assert solve_status.solver.status == SolverStatus.ok

### View Results

In [9]:
# Display Outlet 
m.fs.turbine.outlet.pressure.display()

_outlet_pressure_ref : Size=1, Index=fs.time
    Key : Lower : Value   : Upper        : Fixed : Stale : Domain
    0.0 :   0.1 : 51325.0 : 1000000000.0 : False : False : PositiveReals


In [10]:
# Display a readable report
m.fs.turbine.report()


Unit : fs.turbine                                                          Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                   : Value   : Fixed : Bounds
    Isentropic Efficiency : 0.90000 :  True : (None, None)
          Mechanical Work : -287.43 : False : (None, None)
          Pressure Change : -50000. :  True : (None, None)
           Pressure Ratio : 0.50654 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                  Inlet     Outlet 
    Molar Flow (mol/s)            0.27778   0.27778
    Mass Flow (kg/s)            0.0050042 0.0050042
    T (K)                          390.00    358.29
    P (Pa)                     1.0132e+05    51325.
    Vapor Fraction                 1.0000    1.0000
    Molar Enthalpy (J/mol) Vap     48824.    47789.
    Molar Enthalpy (J/mol) Liq    

In [11]:
import pytest

# Check results
assert m.fs.turbine.outlet.pressure[0].value == pytest.approx(51325, abs=1e-2)